# Importing the libraries

In [24]:
# importing libraries

import pandas as pd
import numpy as np
import matplotlib as plt 
from sklearn.preprocessing import LabelEncoder
import pickle

In [27]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='RCUZbymWi0_Z-RJbNsQus7yG24YUE8Cw_32ZgOmkcyBW',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'carresalevalueprediction-donotdelete-pr-dyf1rlm3gxx4wg'
object_key = 'autos.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_csv(body)
df.head()


,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-24 11:52:17,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


# Cleaning the dataset

In [28]:
df.dtypes

dateCrawled            object
name                   object
seller                 object
offerType              object
price                   int64
abtest                 object
vehicleType            object
yearOfRegistration      int64
gearbox                object
powerPS                 int64
model                  object
kilometer               int64
monthOfRegistration     int64
fuelType               object
brand                  object
notRepairedDamage      object
dateCreated            object
nrOfPictures            int64
postalCode              int64
lastSeen               object
dtype: object

In [29]:
print(df.seller.value_counts())
df[df.seller!='gewerblich']
df=df.drop('seller',1)

print(df.offerType.value_counts())
df[df.offerType!='Gesuch']
df=df.drop('offerType',1)

privat        371525
gewerblich         3
Name: seller, dtype: int64
Angebot    371516
Gesuch         12
Name: offerType, dtype: int64


/tmp/wsuser/ipykernel_164/587171588.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df=df.drop('seller',1)
/tmp/wsuser/ipykernel_164/587171588.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df=df.drop('offerType',1)


In [30]:
print(df.shape)
df=df[(df.powerPS>50) & (df.powerPS)<900]
print(df.shape)
df=df[(df.yearOfRegistration>=1950)&(df.yearOfRegistration<2017)]
print(df.shape)

(371528, 18)
(371528, 18)
(356559, 18)


In [31]:
df.drop(['name','abtest','dateCrawled','nrOfPictures','lastSeen',
'postalCode','dateCreated'],axis='columns',inplace=True)

In [32]:
new_df=df.copy()
new_df=new_df.drop_duplicates(['price','vehicleType','yearOfRegistration',
'gearbox','powerPS','model','kilometer','monthOfRegistration','fuelType',
'notRepairedDamage'])

In [33]:
new_df.gearbox.replace(('manuell','automatik'),('manual', 'automatic'), inplace=True)
new_df.fuelType.replace(('benzin','andere','elektro'),('petrol','others','electric'), inplace=True)
new_df.vehicleType.replace(('kleinwagen','cabrio','kombi','andere'),('small car','convertible','combination','others'),inplace=True)
new_df.notRepairedDamage.replace(('ja','nein'),('Yes','No'),inplace=True)

In [34]:
new_df = new_df[(new_df.price >= 100) & (new_df.price <= 150000)]

new_df['notRepairedDamage'].fillna(value='not-declared', inplace=True)
new_df['fuelType'].fillna(value='not-declared', inplace=True)
new_df['gearbox'].fillna(value='not-declared', inplace=True)
new_df['vehicleType'].fillna(value='not-declared', inplace=True)
new_df['model'].fillna(value='not-declared', inplace=True)

In [35]:
new_df.to_csv('autos_preprocessed.csv')

In [36]:
new_df

,price,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,not-declared,1993,manual,0,golf,150000,0,petrol,volkswagen,not-declared
1,18300,coupe,2011,manual,190,not-declared,125000,5,diesel,audi,Yes
2,9800,suv,2004,automatic,163,grand,125000,8,diesel,jeep,not-declared
3,1500,small car,2001,manual,75,golf,150000,6,petrol,volkswagen,No
4,3600,small car,2008,manual,69,fabia,90000,7,diesel,skoda,No
...,...,...,...,...,...,...,...,...,...,...,...
371523,2200,not-declared,2005,not-declared,0,not-declared,20000,1,not-declared,sonstige_autos,not-declared
371524,1199,convertible,2000,automatic,101,fortwo,125000,3,petrol,smart,No
371525,9200,bus,1996,manual,102,transporter,150000,3,diesel,volkswagen,No
371526,3400,combination,2002,manual,100,golf,150000,6,diesel,volkswagen,not-declared


In [37]:
labels = ['gearbox', 'notRepairedDamage','model','brand','fuelType','vehicleType']

mapper = {}
for i in labels:
  mapper[i] = LabelEncoder()
  mapper[i].fit(new_df[i])
  tr = mapper[i].transform(new_df[i])
  np.save(str('classes'+i+'.npy'), mapper[i].classes_)
  print(i,":",mapper[i])
  new_df.loc[:,i+"_labels"] = pd.Series(tr, index = new_df.index)

labeled = new_df[['price', 'yearOfRegistration','powerPS','kilometer','monthOfRegistration'] + [x+"_labels" for x in labels]]

print(labeled.columns)


gearbox : LabelEncoder()
notRepairedDamage : LabelEncoder()
model : LabelEncoder()
brand : LabelEncoder()
fuelType : LabelEncoder()
vehicleType : LabelEncoder()
Index(['price', 'yearOfRegistration', 'powerPS', 'kilometer',
       'monthOfRegistration', 'gearbox_labels', 'notRepairedDamage_labels',
       'model_labels', 'brand_labels', 'fuelType_labels',
       'vehicleType_labels'],
      dtype='object')


# Splitting Data Into Independent And Dependent Variables

In [38]:
Y = labeled.iloc[:,0].values
X = labeled.iloc[:,1:].values

Y = Y.reshape(-1,1)

In [39]:
from sklearn.model_selection import cross_val_score, train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=3)

In [40]:
X_train,X_test,Y_train,Y_test

(array([[  2009,    101,  40000, ...,     36,      7,      4],
        [  1998,    115, 150000, ...,     10,      5,      1],
        [  2003,    109, 150000, ...,      1,      7,      7],
        ...,
        [  2005,    209, 150000, ...,     39,      7,      8],
        [  2007,    143, 150000, ...,      2,      7,      4],
        [  1999,    136, 150000, ...,      2,      7,      4]]),
 array([[  2006,    140, 100000, ...,     24,      7,      4],
        [  2001,    179, 150000, ...,      1,      1,      1],
        [  1999,    211, 150000, ...,     24,      7,      1],
        ...,
        [  2003,    113, 150000, ...,     27,      4,      7],
        [  1998,    140, 150000, ...,     39,      7,      4],
        [  1994,     75, 150000, ...,     38,      7,      2]]),
 array([[ 7499],
        [  450],
        [ 2990],
        ...,
        [10500],
        [ 6995],
        [ 1899]]),
 array([[5990],
        [2999],
        [ 899],
        ...,
        [2700],
        [ 850],
    

# Building the model

In [41]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

regressor = RandomForestRegressor(n_estimators=1000, max_depth=10, random_state=34)

regressor.fit(X_train, np.ravel(Y_train, order='C'))

RandomForestRegressor(max_depth=10, n_estimators=1000, random_state=34)

In [43]:
y_pred = regressor.predict(X_test)

print(r2_score(Y_test, y_pred))

0.8191322832483275


In [45]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
"url": "https://us-south.ml.cloud.ibm.com",
"apikey":"2AqJ5Gvk1-z-amcRUmp2-UI3xLkmvtIOANNWeCb0EbLB"
}
client = APIClient(wml_credentials)

In [46]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [48]:
space_uid = guid_from_space_name(client,'carResale-model')
print("Space UID = "+space_uid)

Space UID = 503665f6-6bed-4fc2-9bc5-2db4ca82a51c


In [49]:
client.set.default_space(space_uid)

'SUCCESS'

In [50]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [51]:
software_spec_uid = client.software_specifications.get_id_by_name("runtime-22.1-py3.9")
software_spec_uid

'12b83a17-24d8-5082-900f-0ab31fbfd3cb'

In [52]:
model_details = client.repository.store_model(model=regressor,meta_props={
client.repository.ModelMetaNames.NAME:"CarResale_Model",
client.repository.ModelMetaNames.TYPE:"scikit-learn_1.0",
client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid}
)
model_id = client.repository.get_model_id(model_details)

In [53]:
model_id

'b30c835f-df5e-4c24-932f-229dabaa5c82'